In [1]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate
import pandas as pd

In [2]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [3]:
!./setuprds.sh
!./setupdynamo.sh

Creating the DynamoDB table that holds the client floats
{
    "StackId": "arn:aws:cloudformation:us-east-1:123456789:stack/client-float-table/d93ff508-25e8-40a6-8719-531ecf55fd9a"
}
Adding in item for core, ZAR wholesale float


In [4]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: Creating deployment bucket pluto.user.action.lambdas
Serverless: Uploading CloudFormation file to S3...
Serverless: Uploading artifacts...
Serverless: Uploading service user-activity-api.zip file to S3 (27.17 MB)...
Serverless: Validating template...
Serverless: Skipping template validation: Unsupported in Localstack
Serverless: Creating Stack...
Serverless: Checking Stack create progress...
..
Serverless: Stack create finished...
Service Information
service: user-activity-api
stage: local
region: eu-west-1
stack: user-activity-api-local
resources: 9
api keys:
  None
endpoints:
  POST - https://58822A-Z1033.execute-api.eu-west-1.amazonaws.com/local/save
functions:
  save: add-savings
layers:
  None
Serverless: Recoverable error occurred (Inaccessible host: `logs.eu-west-1.amazonaws.com'. This service may not be available in the `eu-west-1' region.), sleeping for 5 seconds. Try 1 of 4
Serverle

In [5]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [6]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [16]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'floatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [17]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [18]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='create-account', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [19]:
def generate_saving_transaction(account_id, ref_amount = 100, float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id
    }
    return test_saving_dict

In [20]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='add-savings', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Tx for account %s persisted with new balance %s' % (tx['accountId'], persisted_tx['currentBalance']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [7]:
print_deployed_functions()

Function list:  ['float-api-local-accrue', 'create-account', 'add-savings']


In [21]:
accounts = create_number_accounts(number_accounts = 10)

Account with ID 608fcab1-73dd-49e0-9a00-43e3fcc61b7d persisted at 2019-06-21T17:03:55.221Z
Account with ID 27c5209b-41b5-4884-87f3-92742e5e3888 persisted at 2019-06-21T17:03:55.716Z
Account with ID d1ece789-3c78-4d65-81a8-5759137acc16 persisted at 2019-06-21T17:03:56.258Z
Account with ID 605e543b-08db-4f58-8355-d7d457db9942 persisted at 2019-06-21T17:03:56.730Z
Account with ID 5956efab-307a-4976-8f70-40c14b933920 persisted at 2019-06-21T17:03:57.230Z
Account with ID f1c91b93-5a3c-4dd1-bf33-e53eac7cc016 persisted at 2019-06-21T17:03:57.797Z
Account with ID c42b1c68-bc79-464a-a7a9-baf9bc933ebf persisted at 2019-06-21T17:03:58.375Z
Account with ID 256e01d2-1e93-4a71-b81d-f478c386ee37 persisted at 2019-06-21T17:03:58.906Z
Account with ID 6772dd6a-81dd-4ffd-ba95-66c03f8584b6 persisted at 2019-06-21T17:03:59.431Z
Account with ID 8b057c21-9b6e-454b-827b-edda7bf1940b persisted at 2019-06-21T17:03:59.966Z


In [22]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts[:100]]
print(tabulate.tabulate(account_info, headers = 'keys'))

Name                AccountId
------------------  ------------------------------------
Barbara Hartness    608fcab1-73dd-49e0-9a00-43e3fcc61b7d
Belinda Howell      27c5209b-41b5-4884-87f3-92742e5e3888
Bertha Rosa         d1ece789-3c78-4d65-81a8-5759137acc16
Constance Fletcher  605e543b-08db-4f58-8355-d7d457db9942
Georgia Kueny       5956efab-307a-4976-8f70-40c14b933920
Steven Butler       f1c91b93-5a3c-4dd1-bf33-e53eac7cc016
William Bostian     c42b1c68-bc79-464a-a7a9-baf9bc933ebf
Annie Turner        256e01d2-1e93-4a71-b81d-f478c386ee37
Julia Reier         6772dd6a-81dd-4ffd-ba95-66c03f8584b6
Betty Erickson      8b057c21-9b6e-454b-827b-edda7bf1940b


In [23]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='add-savings', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

Lambda result {'body': '{"transactionDetails":[[{"transaction_id":"21368a1a-9621-4207-9032-9e6bf0b88dce","creation_time":"2019-06-21T17:04:20.330Z"}],[{"transaction_id":"d6ea7810-29a8-488f-924e-c7361eae579f","creation_time":"2019-06-21T17:04:20.330Z"},{"transaction_id":"d6ea7810-29a8-488f-924e-c7361eae579f","creation_time":"2019-06-21T17:04:20.330Z"}]],"newBalance":508}', 'statusCode': 200}


In [24]:
account_balances = seed_savings_for_accounts(accounts[:100], tx_per_account = 3, base_amount = 100 * 100 * 100)

Tx for account 608fcab1-73dd-49e0-9a00-43e3fcc61b7d persisted with new balance 23729272
Tx for account 27c5209b-41b5-4884-87f3-92742e5e3888 persisted with new balance 94441307
Tx for account d1ece789-3c78-4d65-81a8-5759137acc16 persisted with new balance 70097762
Tx for account 605e543b-08db-4f58-8355-d7d457db9942 persisted with new balance 74275000
Tx for account 5956efab-307a-4976-8f70-40c14b933920 persisted with new balance 25605754
Tx for account f1c91b93-5a3c-4dd1-bf33-e53eac7cc016 persisted with new balance 96123129
Tx for account c42b1c68-bc79-464a-a7a9-baf9bc933ebf persisted with new balance 91492664
Tx for account 256e01d2-1e93-4a71-b81d-f478c386ee37 persisted with new balance 62482365
Tx for account 6772dd6a-81dd-4ffd-ba95-66c03f8584b6 persisted with new balance 46328481
Tx for account 8b057c21-9b6e-454b-827b-edda7bf1940b persisted with new balance 67698268
Tx for account 608fcab1-73dd-49e0-9a00-43e3fcc61b7d persisted with new balance 104755158
Tx for account 27c5209b-41b5-48

In [25]:
def merge_dataframes(accounts, account_balances):
    df = pd.DataFrame(account_balances)
    df = df[df['currentBalance'] == df.groupby('accountId')['currentBalance'].transform('max')]
    df = df[['accountId', 'currentBalance']]
    df['currentBalance'] = pd.to_numeric(df['currentBalance'])
    df['balanceRand'] = df['currentBalance']/10000
    account_name_df = pd.DataFrame(accounts)[['accountId', 'floatId', 'userFamilyName', 'userFirstName']]
    account_name_df['Holder Name'] = account_name_df['userFirstName'] + ' ' + account_name_df['userFamilyName']
    account_name_df.set_index('accountId')
    df.set_index('accountId')
    merged_df = pd.merge(account_name_df, df)[['accountId', 'floatId', 'Holder Name', 'balanceRand']]
    return merged_df

In [26]:
merged_df = merge_dataframes(accounts, account_balances)

In [27]:
merged_df

,accountId,floatId,Holder Name,balanceRand
0,608fcab1-73dd-49e0-9a00-43e3fcc61b7d,zar_cash_float,Barbara Hartness,11300.7903
1,27c5209b-41b5-4884-87f3-92742e5e3888,zar_cash_float,Belinda Howell,20451.7596
2,d1ece789-3c78-4d65-81a8-5759137acc16,zar_cash_float,Bertha Rosa,15139.4053
3,605e543b-08db-4f58-8355-d7d457db9942,zar_cash_float,Constance Fletcher,16216.7434
4,5956efab-307a-4976-8f70-40c14b933920,zar_cash_float,Georgia Kueny,12051.5018
5,f1c91b93-5a3c-4dd1-bf33-e53eac7cc016,zar_cash_float,Steven Butler,23565.7509
6,c42b1c68-bc79-464a-a7a9-baf9bc933ebf,zar_cash_float,William Bostian,12283.3394
7,256e01d2-1e93-4a71-b81d-f478c386ee37,zar_cash_float,Annie Turner,15519.4452
8,6772dd6a-81dd-4ffd-ba95-66c03f8584b6,zar_cash_float,Julia Reier,15208.4063
9,8b057c21-9b6e-454b-827b-edda7bf1940b,zar_cash_float,Betty Erickson,14792.6937


In [28]:
accrualDict = {
    'clientId': 'zar_client_co',
    'floatId': 'zar_cash_primary',
    'accrualAmount': 100 * 100 * 100,
    'currency': 'ZAR',
    'unit': 'HUNDREDTH_CENT',
    'backingEntityIdentifier': 'tx-id-backing'
}
local_lambda.invoke(FunctionName='float-api-local-accrue', InvocationType='RequestResponse', 
                                           Payload=json.dumps(accrualDict, default=str))

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '2240',
   'access-control-allow-origin': '*',
   'server': 'Werkzeug/0.15.4 Python/3.6.8',
   'date': 'Fri, 21 Jun 2019 17:04:56 GMT'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'Payload': <botocore.response.StreamingBody at 0x7f98cd844b70>}